In [3]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [4]:
# Import ords_prods_merge

ords_prods_merge = pd.read_pickle(r'/Users/davidscheider/anaconda_projects/Instacart Basket Analysis/02 Data/Prepared Data/ords_prods_merge_new_columns.pkl')

In [5]:
# Check shape
ords_prods_merge.shape

(32406041, 26)

In [6]:
# Check head
ords_prods_merge.head()

,Unnamed: 0,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,Customer Type,order_frequency_flag
0,0,2539329,1,1,2,8,NaN,196,1,0,...,Mid-range product,Busiest days,Regular day,Average orders,10,New customer,6.367797,Low spender,20.259259,Non-frequent customer
1,0,2539329,1,1,2,8,NaN,14084,2,0,...,Mid-range product,Busiest days,Regular day,Average orders,10,New customer,6.367797,Low spender,20.259259,Non-frequent customer
2,0,2539329,1,1,2,8,NaN,12427,3,0,...,Low-range product,Busiest days,Regular day,Average orders,10,New customer,6.367797,Low spender,20.259259,Non-frequent customer
3,0,2539329,1,1,2,8,NaN,26088,4,0,...,Low-range product,Busiest days,Regular day,Average orders,10,New customer,6.367797,Low spender,20.259259,Non-frequent customer
4,0,2539329,1,1,2,8,NaN,26405,5,0,...,Low-range product,Busiest days,Regular day,Average orders,10,New customer,6.367797,Low spender,20.259259,Non-frequent customer


In [8]:
# create mean for entire dataframe
ords_prods_merge.groupby('department_id')['order_number'].mean()

department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.213779
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.171095
12    15.887671
13    16.583536
14    16.771241
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64

The results are fairly similar. That means, 1 mio lines of the dataframe is a large enough subset to create meaningful analyses

In [9]:
# create column "max_orders" for loyalty flag
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['customer_id'])['order_number'].transform(np.max)

/var/folders/bz/7hwgnc95287cp83lwy4gfzth0000gn/T/ipykernel_14023/208015118.py:2: FutureWarning: The provided callable <function max at 0x108a12fc0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['customer_id'])['order_number'].transform(np.max)


In [10]:
# create flag
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [11]:
# check if flag worked
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15877395
Loyal customer      10284396
New customer         6244250
Name: count, dtype: int64

In [12]:
# check prices column
ords_prods_merge['prices'].head()

0     9.0
1    12.5
2     4.4
3     4.7
4     1.0
Name: prices, dtype: float64

In [13]:
# check statistics for product prices using loyalty flag
ords_prods_merge.groupby('loyalty_flag')['prices'].describe()

,count,mean,std,min,25%,50%,75%,max
loyalty_flag,,,,,,,,
Loyal customer,10284396.0,10.386229,328.012956,1.0,4.2,7.4,11.2,99999.0
New customer,6244250.0,13.294406,597.547859,1.0,4.2,7.4,11.3,99999.0
Regular customer,15877395.0,12.495501,539.710399,1.0,4.2,7.4,11.3,99999.0


Explanation
New customers have the highest average spending per product ($13.29), followed by regular ($12.50), and then loyal customers ($10.39).
This suggests that loyal customers spend less per item on average than new or regular customers. All three groups have the same median price ($7.4), which suggests that the typical purchase is very similar across groups.

New customers have the highest spending per product (13.29) followed by regular (12.50) and then loyal customers(10.39). This suggests that loyal customers spend less per item on average than new or regular customers.

In [14]:
# create column "avg_price" for spending flag
ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['customer_id'])['prices'].transform(np.mean)

/var/folders/bz/7hwgnc95287cp83lwy4gfzth0000gn/T/ipykernel_14023/4293150670.py:2: FutureWarning: The provided callable <function mean at 0x108a139c0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['customer_id'])['prices'].transform(np.mean)


In [15]:
# create flag
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High spender'

In [16]:
# check if flag worked
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31771789
High spender      634252
Name: count, dtype: int64

There are 31771789 low spenders and 634252 high spenders in this dataframe

In [18]:
# create column "avg_price" for spending flag
ords_prods_merge['Customer Type'] = ords_prods_merge.groupby(['customer_id'])['days_since_prior_order'].transform(np.median)

/var/folders/bz/7hwgnc95287cp83lwy4gfzth0000gn/T/ipykernel_14023/1469485198.py:2: FutureWarning: The provided callable <function median at 0x108b45e40> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['Customer Type'] = ords_prods_merge.groupby(['customer_id'])['days_since_prior_order'].transform(np.median)


In [19]:
# create flag
ords_prods_merge.loc[ords_prods_merge['Customer Type'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['Customer Type'] > 10) & (ords_prods_merge['Customer Type'] <= 20), 'order_frequency_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['Customer Type'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [20]:
# check if flag worked
ords_prods_merge['order_frequency_flag'].value_counts(dropna = False)

order_frequency_flag
Frequent customer        21560324
Regular customer          7209005
Non-frequent customer     3636707
NaN                             5
Name: count, dtype: int64

In [21]:
ords_prods_merge

,Unnamed: 0,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,Customer Type,order_frequency_flag
0,0,2539329,1,1,2,8,NaN,196,1,0,...,Mid-range product,Busiest days,Regular day,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,0,2539329,1,1,2,8,NaN,14084,2,0,...,Mid-range product,Busiest days,Regular day,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,0,2539329,1,1,2,8,NaN,12427,3,0,...,Low-range product,Busiest days,Regular day,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,0,2539329,1,1,2,8,NaN,26088,4,0,...,Low-range product,Busiest days,Regular day,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,0,2539329,1,1,2,8,NaN,26405,5,0,...,Low-range product,Busiest days,Regular day,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32406036,3421081,2977660,206209,13,1,12,7.0,14197,5,1,...,Mid-range product,Busiest days,Busiest days,Average orders,13,Regular customer,7.058915,Low spender,22.0,Non-frequent customer
32406037,3421081,2977660,206209,13,1,12,7.0,38730,6,0,...,Mid-range product,Busiest days,Busiest days,Average orders,13,Regular customer,7.058915,Low spender,22.0,Non-frequent customer
32406038,3421081,2977660,206209,13,1,12,7.0,31477,7,0,...,Low-range product,Busiest days,Busiest days,Average orders,13,Regular customer,7.058915,Low spender,22.0,Non-frequent customer
32406039,3421081,2977660,206209,13,1,12,7.0,6567,8,0,...,Low-range product,Busiest days,Busiest days,Average orders,13,Regular customer,7.058915,Low spender,22.0,Non-frequent customer


In [22]:
# define path

path = r'/Users/davidscheider/anaconda_projects/Instacart Basket Analysis'

In [23]:
# export to pickle file
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data','ords_prods_merge_new_columns.pkl'))